# 2021-04-28 Daily Practice

- [ ] Leedcode / HackerRank algorithms practice
- [x] Build

---

# Savor Dash

Wrote a couple of functions: 

- Functionalized code to create the id-name lookup
- One to filter records to those within a certain time range. As the records in question are in UTC, I had to play around with the Python timezone functionality to get things lined up. 
    - Didn't quite finish this one, but got the dates and filters working at a basic level at least.
    - Also got to use a for-else loop for the first time!

In [ ]:
def create_lookup(base_key: str, api_key: str, table_fields: dict) -> pd.Series:
    """Creates an ID lookup series from a set of Airtable tables."""
    series_index = []  # Index (id) array
    series_data = []  # Series data
    for table in tables_and_fields:  # Loop through tables to retrieve records and save in array
        airtable = Airtable(base_key, table, api_key=api_key)  # Connect to table
        records = airtable.get_all(fields=tables_and_fields[table])  # Retrieve records
        for record in records:  # Transform and save records to arrays
            series_index.append(record["id"])
            series_data.append(record["fields"]["name"])
    # Return series, indexed by ID
    return pd.Series(series_data, index=series_index)

In [ ]:
from datetime import datetime

def get_data_for_date_range(
    conn: Airtable,
    start: datetime,
    end: datetime,
) -> pd.DataFrame:
    """Retrieve Airtable data within the specified time range.
    Assumes that datetimes passed are in UTC, so convert to UTC beforehand."""
    # Loop through pages and records within pages, checking the time_in against the range
    # If inside range, append to list of raw data; else break from loop
    raw_records = []
    for page in conn.get_iter(sort=["-time_in"], page_size=50):
        # Page size of 50 = roughly # records in average day
        for record in page:
            # Airtable saves records in UTC
            time_in = datetime.fromisoformat(record["fields"]["time_in"][:-5] + "+00:00")
            if time_in > end:
                continue
            elif time_in > start and time_in < end:
                raw_records.append(record)
            else:
                break
        else:
            continue
        break  # Break outer loop when inner loop is broken
    
    return raw_records

In [ ]:
import dateutil

# Create start and end UTC datetimes
mdt = dateutil.tz.gettz("America/Denver")  # MDT timezone info
# Convert the MDT date to UTC
start = datetime(2021, 4, 26, tzinfo=mdt).astimezone(timezone.utc)
end = datetime.now(timezone.utc) # Now/today, in UTC

raw_records = get_data_for_date_range(engage_log, start, end)